# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

# Assignment 1 answers Nov 1 2200h

In [29]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [30]:
# Load libraries
import os
from dotenv import load_dotenv
import pandas as pd
import dask.dataframe as dd
from glob import glob
import yfinance as yf
import sys
import numpy as np

sys.path.append(os.getenv('SRC_DIR'))

# Load Dask DataFrame
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [32]:
ft_path = os.getenv("PRICE_DATA")
# df_raw = pd.read_csv(ft_path)
# os.makedirs(ft_path, exist_ok=True)

# Find parquet files
parquet_files = glob(os.path.join(ft_path, "*/path.o.parquet"), recursive=True)

# Print list of parquet files found
print("Parquet files found:", parquet_files)

# Print head and columns of DataFrame
print(dd_price_data.head())
print(dd_price_data.columns)

# Strip whitespace from column names
dd_price_data.columns = dd_price_data.columns.str.strip()

# Check if the 'Price ticker' column exists
if 'ticker' in dd_price_data.columns:
    print("The 'Price ticker' column is present.")
else:
    print("The 'Price ticker' column is missing.")

Parquet files found: []
Price                       Date  Adj Close      Close       High        Low  \
ticker                                                                         
A      2000-01-03 00:00:00+00:00  43.463020  51.502148  56.464592  48.193848   
A      2000-01-04 00:00:00+00:00  40.142944  47.567955  49.266811  46.316166   
A      2000-01-05 00:00:00+00:00  37.652874  44.617310  47.567955  43.141991   
A      2000-01-06 00:00:00+00:00  36.219196  42.918453  44.349072  41.577251   
A      2000-01-07 00:00:00+00:00  39.237442  46.494991  47.165592  42.203148   

Price        Open   Volume       sector                       subsector  year  \
ticker                                                                          
A       56.330471  4674353  Health Care  Life Sciences Tools & Services  2000   
A       48.730328  4765083  Health Care  Life Sciences Tools & Services  2000   
A       47.389126  5758642  Health Care  Life Sciences Tools & Services  2000   
A       44

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [28]:
# Load the data from Parquet files
dd_price_data = dd.read_parquet(r"..\..\05_src\data\prices")

# Check if 'Adj_Close' exists, if not, create it
if 'Adj_Close' not in dd_price_data.columns:
    if 'Close' in dd_price_data.columns:
        dd_price_data['Adj_Close'] = dd_price_data['Close']  # Adjust as needed
    else:
        raise KeyError("Missing required column: Close to create Adj_Close")

# Define a function to process each ticker separately
def process_ticker(data):
    # Check if required columns exist
    required_columns = ['Close', 'Adj_Close', 'High', 'Low', 'ticker']
    for col in required_columns:
        if col not in data.columns:
            raise KeyError(f"Missing required column: {col}")

    # Create lagged variables for Close and Adj_Close
    data['Close_lag'] = data['Close'].shift(1)
    data['Adj_Close_lag'] = data['Adj_Close'].shift(1)
    
    # Calculate returns based on Adj_Close
    data['returns'] = (data['Adj_Close'] / data['Adj_Close_lag']) - 1
    
    # Calculate hi_lo_range 
    data['hi_lo_range'] = data['High'] - data['Low']
    
    return data

# Group by 'ticker' and apply the function
dd_feat = dd_price_data.groupby('ticker').apply(process_ticker, meta={
    'Date': 'datetime64[ns]',
    'Price ticker': 'object',
    'Close': 'float64',
    'Adj_Close': 'float64',
    'Close_lag': 'float64',
    'Adj_Close_lag': 'float64',
    'returns': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'hi_lo_range': 'float64'
})

# Show sample of the result 
print(dd_feat.compute())

c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\pyarrow\pandas_compat.py:766: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.
  klass=_int.DatetimeTZBlock,
c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


KeyError: 'Missing required column: ticker'

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [18]:
# Convert to pandas DataFrame
df_pandas = dd_feat.compute()

# Add a rolling average return calculation with a 10-day window
df_pandas['rolling_avg_return'] = df_pandas['returns'].rolling(window=10).mean()

KeyError: 'Missing required column: ticker'

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1. Was it necessary to convert to pandas to calculate the moving average return?
# No, it wasn't strictly necessary. Dask provides functionality for rolling operations including moving averages, which could have been used directly on the Dask DataFrame.

2. Would it have been better to do it in Dask? Why?
# It would have been better to use Dask for a number of reasons, including the following:

# Scalability: Dask is designed to handle larger-than-memory datasets. If the dataset is very large, Dask can process it in chunks without loading everything into memory at once.

# Parallel processing: Dask can leverage multiple cores or even a cluster of machines to perform computations in parallel, potentially speeding up the calculation for large datasets.

# Lazy evaluation: Dask uses lazy evaluation, which means it builds up a task graph of operations and only executes when necessary, which makes running the computation more efficient especially for complex chains of operations.

# Consistency: Keeping the data in Dask format throughout the pipeline maintains consistency and allows for easier integration with other Dask operations.

# Large datasets: Dask can be more useful and run more efficiently for large datasets

# In this specific case, given that we're dealing with financial time series data that could potentially be quite large, it would likely have been better to perform the rolling average calculation in Dask

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.